<a href="https://colab.research.google.com/github/tonioyoon/llama2-test/blob/main/LLAM2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* https://www.youtube.com/watch?v=BzHfLzP-gfg
  * https://github.com/NajiAboo/LLMFinetune/tree/main/datasetpreprocessing

# Google Drive 연결

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
%cd drive/MyDrive/AI/LLAMA2/

[Errno 2] No such file or directory: 'drive/MyDrive/AI/LLAMA2/'
/content/drive/MyDrive/AI/LLAMA2


# Dataset 준비

* 허깅페이스로 부터 데이터를 받는다.
* 요건 데이터셋을 이렇게 만든다 정도이다. 실제 나중에는 다른 데이터셋을 사용한다.
* 사용하는 데이터를 보면 이 포멧도 아닌것 같다. --;

In [ ]:
!pip install datasets

In [ ]:
from datasets import load_dataset
dataset = load_dataset("gamino/wiki_medical_terms")

for split, dataset in dataset.items():
    dataset.to_json(f"wiki_medical_terms_{split}.jsonl")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Creating json from Arrow format:   0%|          | 0/7 [00:00<?, ?ba/s]

* 받은 데이터를 fine tune 포멧으로 변경한다.

In [ ]:
import json

def format_entry(entry):
    formatted_entry = {
        "text": f"<s> [INST] <<SYS>> You are a helpful, respectful, and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature. If a question does not make any sense or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information. <</SYS>> What is {entry['page_title']} and explain in detail? [/INST] {entry['page_text']} </s>"
    }
    return formatted_entry

formatted_data = []

with open("wiki_medical_terms_train.jsonl","r") as wiki:
    for line in wiki:
        entry = json.loads(line)
        #remove unwanted references
        cut_lasttwolines = entry["page_text"].split(". ")[:-10]
        entry["page_text"] = ". ".join(cut_lasttwolines)
        formatted_entry = format_entry(entry=entry)
        formatted_data.append(formatted_entry)

print(formatted_data)
with open("wiki_medical_terms_llam2.jsonl","w") as llama2:
    for entry in formatted_data:
        try:
            json.dump(entry, llama2)
            llama2.write('\n')
        except Exception as ex:
            print(ex)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



# LLmas2 fine tuning - Training

In [ ]:
!pip install huggingface_hub
!pip install autotrain-advanced
!pip install datasets

In [4]:
from datasets import load_dataset
dataset = load_dataset('aboonaji/alpaca_micro_demo', split='train')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [5]:
dataset

Dataset({
    features: ['instruction', 'input', 'output', 'text'],
    num_rows: 120
})

In [7]:
!autotrain llm\
 --train\
 --project_name llam\
 --model NousResearch/Llama-2-7b-chat-hf\
 --data_path aboonaji/alpaca_micro_demo\
  --use_peft\
  --use_int4\
  --learning_rate 2e-4 \
  --num_train_epochs 1\
  --trainer sft

> INFO    Running LLM
> INFO    Params: Namespace(version=False, train=True, deploy=False, inference=False, data_path='aboonaji/alpaca_micro_demo', train_split='train', valid_split=None, text_column='text', rejected_text_column='rejected', prompt_text_column='prompt', model='NousResearch/Llama-2-7b-chat-hf', model_ref=None, learning_rate=0.0002, num_train_epochs=1, train_batch_size=2, warmup_ratio=0.1, gradient_accumulation_steps=1, optimizer='adamw_torch', scheduler='linear', weight_decay=0.0, max_grad_norm=1.0, seed=42, add_eos_token=False, block_size=-1, use_peft=True, lora_r=16, lora_alpha=32, lora_dropout=0.05, logging_steps=-1, project_name='llam', evaluation_strategy='epoch', save_total_limit=1, save_strategy='epoch', auto_find_batch_size=False, fp16=False, push_to_hub=False, use_int8=False, model_max_length=1024, repo_id=None, use_int4=True, trainer='sft', target_modules=None, merge_adapter=False, token=None, backend='default', username=None, use_flash_attention_2=False, log='n

In [10]:
! ls ./llam

adapter_config.json  checkpoint-6	      tokenizer_config.json  training_args.bin
adapter_model.bin    README.md		      tokenizer.json	     training_params.json
added_tokens.json    special_tokens_map.json  tokenizer.model


#LLmas2 fine tuning - Inference

In [ ]:
!pip install transformers
!pip install accelerate

In [14]:
from transformers import AutoTokenizer
import transformers
import torch

model = "./llam"

tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [17]:
sequences = pipeline(
    'Give three tips for staying healthy\n',
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_length=200,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Result: Give three tips for staying healthy

Healthy habits are essential for staying healthy, but they can be difficult to remember and implement. Here are three tips for staying healthy, backed by research:

1. Eat a healthy diet: A healthy diet provides the body with the nutrients it needs to function optimally. Aim to eat a variety of fruits, vegetables, whole grains, lean proteins, and healthy fats. Limit your intake of processed foods, added sugars, and saturated fats.
2. Get enough sleep: Sleep is essential for overall health, and research has shown that sleep deprivation can have serious consequences for your physical and mental health. Aim for 7-9 hours of sleep each night, and establish a consistent sleep schedule to help regulate your body's internal clock.
3. Exercise regularly
